# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921682407557                   -0.69    6.0         
  2   -7.926162632183       -2.35       -1.22    1.0    308ms
  3   -7.926837575178       -3.17       -2.37    1.9    365ms
  4   -7.926861501142       -4.62       -3.04    2.6    404ms
  5   -7.926861649934       -6.83       -3.43    1.9    373ms
  6   -7.926861671354       -7.67       -3.84    1.5    315ms
  7   -7.926861680206       -8.05       -4.29    1.2    288ms
  8   -7.926861681756       -8.81       -4.87    1.9    327ms
  9   -7.926861681858       -9.99       -5.20    1.9    363ms
 10   -7.926861681871      -10.88       -5.87    1.4    315ms
 11   -7.926861681873      -11.84       -6.49    2.2    347ms
 12   -7.926861681873      -13.14       -7.39    2.1    384ms
 13   -7.926861681873   +  -14.35       -8.01    2.5    346ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921712526243                   -0.69    5.8         
  2   -7.926164962620       -2.35       -1.22    1.0    382ms
  3   -7.926837728298       -3.17       -2.37    1.8    357ms
  4   -7.926861524732       -4.62       -3.02    2.6    388ms
  5   -7.926861639007       -6.94       -3.37    1.8    311ms
  6   -7.926861668200       -7.53       -3.75    1.8    374ms
  7   -7.926861680607       -7.91       -4.38    1.1    285ms
  8   -7.926861681802       -8.92       -4.94    2.2    340ms
  9   -7.926861681865      -10.21       -5.36    1.9    337ms
 10   -7.926861681872      -11.13       -6.07    1.9    357ms
 11   -7.926861681873      -12.15       -7.05    2.0    353ms
 12   -7.926861681873      -14.01       -7.61    3.0    414ms
 13   -7.926861681873   +  -15.05       -8.32    2.0    344ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921667660050                   -0.69    5.9         
  2   -7.926166849965       -2.35       -1.22    1.0    300ms
  3   -7.926839887578       -3.17       -2.37    1.6    335ms
  4   -7.926864900252       -4.60       -3.01    2.9    433ms
  5   -7.926865045745       -6.84       -3.34    1.9    306ms
  6   -7.926865077600       -7.50       -3.73    1.6    300ms
  7   -7.926865091673       -7.85       -4.38    1.0    267ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
